<a href="https://colab.research.google.com/github/CristinaGHolgado/old-french-lemmatization/blob/master/train_tag_udpipe_old_french.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("udpipe")

R[write to console]: 

R[write to console]: 
R[write to console]: The downloaded source packages are in
	‘/tmp/RtmpsVOvkW/downloaded_packages’
R[write to console]: 
R[write to console]: 



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir("gdrive/MyDrive/Lemmatisation AF") # path to folders
!ls

TEST_1			  TEST_3_tagged_cc_udpipe  TEST_6_udpipe_model
TEST_10			  TEST_3_udpipe_model	   TEST_7
TEST_10_tagged_cc_udpipe  TEST_4		   TEST_7_tagged_cc_udpipe
TEST_10_udpipe_model	  TEST_4_tagged_cc_udpipe  TEST_7_udpipe_model
TEST_1_tagged_cc_udpipe   TEST_4_udpipe_model	   TEST_8
TEST_1_udpipe_model	  TEST_5		   TEST_8_tagged_cc_udpipe
TEST_2			  TEST_5_tagged_cc_udpipe  TEST_8_udpipe_model
TEST_2_tagged_cc_udpipe   TEST_5_udpipe_model	   TEST_9
TEST_2_udpipe_model	  TEST_6		   TEST_9_tagged_cc_udpipe
TEST_3			  TEST_6_tagged_cc_udpipe  TEST_9_udpipe_model


## Run for training

In [ ]:
import glob
path_list = [os.path.join(dirpath,filename) for dirpath, _, filenames in os.walk('.') for filename in filenames if filename.endswith('UDPIPE_ca_final.conllu')]
path_list

[]

In [ ]:
%%R
library(udpipe)
CA_files <- list.files(pattern = "UDPIPE_ca_final.conllu$", recursive = TRUE)

params <- list()

params$tagger <- list(models = 2, 
  templates_1 = "tagger", 
      guesser_suffix_rules_1 = 8, guesser_enrich_dictionary_1 = 6, 
      guesser_prefixes_max_1 = 0, 
      use_lemma_1 = 1, use_xpostag_1 = 1, use_feats_1 = 1, 
      provide_lemma_1 = 0, provide_xpostag_1 = 1, 
      provide_feats_1 = 1, prune_features_1 = 0, 
  templates_2 = "lemmatizer", 
      guesser_suffix_rules_2 = 2, guesser_enrich_dictionary_2 = 4, 
      guesser_prefixes_max_2 = 4, 
      use_lemma_2 = 1, use_xpostag_2 = 1, use_feats_2 = 1, 
      provide_lemma_2 = 1, provide_xpostag_2 = 0, 
      provide_feats_2 = 0, prune_features_2 = 0)

train_annotate <- function(traindata) {
    traindata <- traindata
    modelname <- paste(strsplit(traindata, "/")[[1]][1],"_udpipe_model")
    train <- udpipe_train(file = modelname, files_conllu_training = traindata, 
                          annotation_tokenizer = list(dimension = 24, epochs = 100, initialization_range = 0.1, batch_size = 100, learning_rate=0.005, dropout = 0.1, early_stopping = 1),
                          annotation_tagger = params$tagger,
                          annotation_parser = "none")
}

for(i in CA_files){train_annotate(i)}


## Run for tagging

In [ ]:
%%R
library(stringi)
library(udpipe)
ccontrole_files <- list.files(pattern = "corpus_controle_final.csv$", recursive = TRUE, full.names=TRUE)
models_udpipe <- list.files(pattern = "_udpipe_model",recursive=TRUE, full.names=FALSE)

d <- data.frame(cc = ccontrole_files, mod= models_udpipe)
d$x <- paste(d$cc, "--", d$mod)
d <- d$x

tag_udpipe <- function(cc_file, model){
    load_model <- udpipe_load_model(file = model)
    corpuscontrole <- paste(readLines(cc_file))
     annotated <- udpipe_annotate(load_model, x = corpuscontrole, tokenizer = "vertical", tagger = "default", parser = "none", trace = FALSE)
     df <- as.data.frame(annotated)
     outputname = gsub("[[:blank:]]","", paste(strsplit(model,"_udpipe_model")[[1]][1],"_tagged_cc_udpipe"))
     write.table(df[6:10], outputname, sep='\t', fileEncoding = "UTF-8", quote=FALSE,  col.names=FALSE, row.names=FALSE)
}

for(i in d){
    cc = gsub("[[:blank:]]","", strsplit(i,"--")[[1]][1])
    models = gsub("[[:blank:]]","", strsplit(i, "--")[[1]][2])
    tag_udpipe(cc, models)
}